encoder transformer层的linear1层（etl1）替换为近似矩阵乘法

In [1]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE

In [3]:
feedback_bits = 256
ncodebooks = 64 # max:64
ncentroids = 256
if method == METHOD_MITHRAL:
    ncentroids = 16
train_sam_num = 1000 # 训练集样本数

In [4]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= ''
    featurepath_train= ''
    data_to_fcpath_test = ''
    featurepath_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'ex_linear1in_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'ex_linear1_y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'ex_linear1out_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'ex_linear1in_test.npy'
    featurepath_test = 'ex_linear1out_test.npy'
    # y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'ex_linear1_w_f256.npy'
biaspath = 'ex_linear1_b_f256.npy'
dir_result = os.path.join(dir_result, method)


In [5]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  Mithral
X_res mse / X mse:  0.005530617
fitting dense lstsq to X_res
X_res mse / X mse after lstsq:  0.0047106366


In [6]:
type(est3)

vq_amm.MithralMatmul

In [7]:
x_test = np.load(dir_test+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [8]:
print(y_out_last)
print(y_out_last.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(dir_result+'/'+method+'etl1_sam%i_fb%i_cb%i_ct%i.npy' % (train_sam_num, feedback_bits, ncodebooks, ncentroids), y_out_last)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[-0.53261286 -0.42106825 -0.5153788  ... -0.41666412  1.3373108
  -0.4653702 ]
 [-0.53261286 -0.42106825 -0.5153788  ... -0.41666412  1.2120662
  -0.4653702 ]
 [-0.4073682  -0.42106825 -0.39013416 ... -0.29141948  1.2120662
  -0.3401256 ]
 ...
 [-0.15687898 -0.17057902 -0.26488954 ... -0.29141948  0.7110877
  -0.3401256 ]
 [-0.2821236  -0.29582363 -0.5153788  ... -0.04093025  0.96157694
  -0.3401256 ]
 [-0.2821236  -0.29582363 -0.1396449  ... -0.04093025  0.5858431
  -0.21488097]]
(1024000, 512)
